In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np 
import os
import sqlite3
from time import sleep

In [2]:
API_KEY = os.getenv("API_KEY")
watcher = LolWatcher(API_KEY)
region = 'NA1' 

In [3]:
connection = sqlite3.connect("riot_matches2.db")
create_match_table = """CREATE TABLE IF NOT EXISTS match (
id INTEGER PRIMARY KEY,
game_id TEXT,
blue_dragons INTEGER,
blue_heralds INTEGER,
blue_barons INTEGER,
blue_turrets INTEGER,
blue_kills INTEGER,
blue_gold_20 INTEGER,
red_dragons INTEGER,
red_heralds INTEGER,
red_barons INTEGER,
red_turrets INTEGER,
red_kills INTEGER,
red_gold_20 INTEGER,
winner TEXT
)"""

cur = connection.cursor()
cur.execute(create_match_table)
insert_match = "INSERT OR IGNORE INTO match (game_id, blue_dragons, blue_heralds, blue_barons, blue_turrets, blue_kills, blue_gold_20, red_dragons, red_heralds, red_barons, red_turrets, red_kills, red_gold_20, winner) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"

In [4]:
challenger_dto = watcher.league.challenger_by_queue(region, queue="RANKED_SOLO_5x5")
challenger_players = challenger_dto["entries"]

grandmaster_dto = watcher.league.grandmaster_by_queue(region, queue="RANKED_SOLO_5x5")
grandmaster_players = grandmaster_dto["entries"]

master_dto = watcher.league.masters_by_queue(region, queue="RANKED_SOLO_5x5")
master_players = master_dto["entries"]

In [5]:
high_elo_players = []
for summoner in challenger_players:
    high_elo_players.append(summoner["summonerName"])
    
for summoner in grandmaster_players:
    high_elo_players.append(summoner["summonerName"])

for summoner in master_players:
    high_elo_players.append(summoner["summonerName"])
    
print("Length: ", len(high_elo_players))
high_elo_players[0:5]

Length:  6453


['IIrrooGG', 'zi du', 'Douzheng', 'AMT', 'Dantes']

In [6]:
def get_summoner_uid(name):
    try:
        summoner = watcher.summoner.by_name(region, name)
    except ApiError:
        return "No summoner found"
    return summoner["puuid"]

In [7]:
def get_data_from_match(player_uid):
    
    match_list = watcher.match.matchlist_by_puuid('americas', player_uid, queue=420, type="ranked")
    
    for match in match_list:
        game_timeline = watcher.match.timeline_by_match('americas', match)
        game_details = watcher.match.by_id('americas', match)
        if game_details["info"]["participants"][0]["win"] == True:
            winner = "Team 1"
        else:
            winner = "Team 2"
                
       
        if len(game_timeline['info']['frames']) < 22:
            # print("less than 20")
            continue 
        # print("more than 20")
        frames =  game_timeline['info']['frames'][:22]
        
        # get gold diff 
        gold = {}
        for k in range(len(frames)):
            gold['{}'.format(k)] = {}
        for value in gold.values():
            for l in range(1, 11):
                value['{}'.format(l)] = 0

        for m in range(len(frames)):
            for n in range(1, 11):
                gold['{}'.format(m)]['{}'.format(n)] = game_timeline['info']['frames'][m]['participantFrames']['{}'.format(n)]['totalGold']

        blue_gold = []
        red_gold = []
        
        for key in gold.keys():
            team1 = 0
            team2 = 0
            for o in range(1, 6):
                team1 += gold[key]['{}'.format(o)]
                team2 += gold[key]['{}'.format(o + 5)]
            blue_gold.append(team1)
            red_gold.append(team2)

        # gold_diff = (np.array(team_1_gold) - np.array(team_2_gold)).tolist()
        
        
        blue_dragons = 0
        blue_heralds = 0
        red_dragons = 0
        red_heralds = 0
        blue_barons = 0
        red_barons = 0

        for frame in frames:
            for event in frame["events"]:
                if "monsterType" in event.keys():
                    if event["monsterType"] == "DRAGON":
                        if event["killerTeamId"] == 100:
                            blue_dragons += 1
                        else:
                            red_dragons += 1
                    if event["monsterType"] == "RIFTHERALD":
                        if event["killerTeamId"] == 100:
                            blue_heralds += 1
                        else:
                            red_heralds += 1
                    if event["monsterType"] == "BARON_NASHOR":
                        if event["killerTeamId"] == 100:
                            blue_barons += 1
                        else:
                            red_barons += 1
          
        # print(match, blue_gold[-1], red_gold[-1], blue_dragons, blue_heralds, blue_barons, red_dragons, red_heralds, red_barons)
        
        blue_k = 0 # blue side (bottom)
        red_k = 0 # red side (top)

        for frame in frames:
            for event in frame["events"]:
                if "victimId" in event.keys():
                    if event["type"] == "CHAMPION_KILL":
                        if event["killerId"] == 0:
                            continue
                        if 1 <= event["killerId"] <= 5:
                            blue_k += 1
                        else:
                            red_k += 1
                            
        blue_obj = 0
        red_obj = 0
        for frame in frames:
            for event in frame["events"]:
                if "buildingType" in event.keys():
                    if event["teamId"] == 100:
                        red_obj += 1
                    else:
                        blue_obj += 1
        
        # print(match, blue_dragons, blue_heralds, blue_barons, blue_obj, blue_k, blue_gold[-1], red_dragons, red_heralds, red_barons, red_obj, red_k, red_gold[-1], winner)
        cur.execute(insert_match, (match, blue_dragons, blue_heralds, blue_barons, blue_obj, blue_k, blue_gold[-1], red_dragons, red_heralds, red_barons, red_obj, red_k, red_gold[-1], winner))
        connection.commit()
        sleep(0.84)
        

In [8]:
for name in high_elo_players:
    player_uid = get_summoner_uid(name)
    if player_uid == "No summoner":
        continue
    try:
        get_data_from_match(player_uid)
    except:
        continue

In [ ]:
print("finished gathering data")